📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/pytorch-workshop](https://github.com/mr-pylin/pytorch-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Custom Classes in PyTorch](#toc2_)    
  - [Load Breast Cancer Wisconsin (Diagnostic) Dataset](#toc2_1_)    
  - [Custom Dataset](#toc2_2_)    
  - [Custom Transform](#toc2_3_)    
    - [Direct transform](#toc2_3_1_)    
    - [Integrated transform](#toc2_3_2_)    
  - [Custom Activation Function](#toc2_4_)    
  - [Custom Model](#toc2_5_)    
  - [Custom Loss Function](#toc2_6_)    
  - [Custom Optimizer](#toc2_7_)    
- [Example: All In One](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import transforms

# <a id='toc2_'></a>[Custom Classes in PyTorch](#toc0_)

- **PyTorch** is a **flexible** deep learning framework that allows developers to **customize** different components according to their **specific needs**.
- This flexibility is essential for **implementing** custom **datasets**, **models**, and **optimization** routines, which may not be covered by the **built-in classes**.

---


## <a id='toc2_1_'></a>[Load Breast Cancer Wisconsin (Diagnostic) Dataset](#toc0_)


In [ ]:
breast_cancer_dataset_url = r"https://github.com/mr-pylin/datasets/raw/refs/heads/main/data/tabular-data/breast-cancer-wisconsin-diagnostic/dataset.csv"

# pandas data-frame
df = pd.read_csv(breast_cancer_dataset_url, encoding="utf-8")

# encode labels
df["Diagnosis"] = df["Diagnosis"].map({"B": 0, "M": 1})

# log
df.head()

In [ ]:
classes = df["Diagnosis"].unique()
class_to_idx = {l: i for i, l in enumerate(classes)}

# split dataset into features and labels
X, y = df.iloc[:, 2:].values, df.iloc[:, 1].values

# convert categorical labels into indices
y = np.array([class_to_idx[l] for l in y])

# properties of the dataset
num_samples, num_features = X.shape
classes, samples_per_class = np.unique(y, return_counts=True)

# log
print(f"X.shape: {X.shape}")
print(f"X.dtype: {X.dtype}")
print(f"y.shape: {y.shape}")
print(f"y.dtype: {y.dtype}")
print("-" * 50)
print(f"classes          : {classes}")
print(f"samples per class: {samples_per_class}")

## <a id='toc2_2_'></a>[Custom Dataset](#toc0_)

- PyTorch’s `Dataset` class can be easily subclassed to define custom datasets
- This allows you to load and preprocess your data according to your needs.
- Use `torch.utils.data.Dataset` as the parent class and override `__len__` and `__getitem__`.

📝 **Docs & Tutorials** 📚:

- Data Loading Utility: [pytorch.org/docs/stable/data.html](https://pytorch.org/docs/stable/data.html)
- Datasets & DataLoaders: [pytorch.org/tutorials/beginner/basics/data_tutorial.html](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)


In [ ]:
# convert numpy.ndarray to torch.Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# log
print(f"type(X): {type(X)}  |  X.dtype: {X.dtype}  |  X.shape: {X.shape}")
print(f"type(y): {type(y)}  |  y.dtype: {y.dtype}  |  y.shape: {y.shape}")

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data: torch.Tensor, labels: torch.Tensor) -> None:
        self.data = data
        self.labels = labels

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.data[index], self.labels[index]

In [ ]:
# create a pytorch dataset
dataset_1 = CustomDataset(X, y)  # custom
dataset_2 = TensorDataset(X, y)  # built-in

# log
print(f"type(dataset_1) : {type(dataset_1)}")
print(f"len(dataset_1)  : {len(dataset_1)}")
print(f"dataset_1[0]    : {dataset_1[0]}")
print("-" * 50)
print(f"type(dataset_2) : {type(dataset_2)}")
print(f"len(dataset_2)  : {len(dataset_2)}")
print(f"dataset_2[0]    : {dataset_2[0]}")

## <a id='toc2_3_'></a>[Custom Transform](#toc0_)

- Transforms are used to modify the input data before feeding it into the model.
- PyTorch provides a lot of built-in transforms (like cropping, flipping, etc.) in `torchvision.transforms`.
- you can define your own transformation by implementing the `__call__` method.

📝 **Docs & Tutorials** 📚:

- Transforming and augmenting images: [pytorch.org/vision/stable/transforms.html](https://pytorch.org/vision/stable/transforms.html)
- Transforms: [pytorch.org/tutorials/beginner/basics/transforms_tutorial.html](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html)


In [7]:
class NumpyToTensor:
    def __call__(self, sample: np.ndarray) -> tuple[torch.Tensor, torch.Tensor]:
        converted_sample = torch.tensor(sample[0], dtype=torch.float32), torch.tensor(sample[1], dtype=torch.float32)
        return converted_sample

In [ ]:
class NormalizeTo01:
    def __init__(self) -> None:
        self.min_val = None
        self.max_val = None

    def fit(self, data: np.ndarray) -> None:
        self.min_val = np.min(data, axis=0).astype(np.float32)
        self.max_val = np.max(data, axis=0).astype(np.float32)

    def __call__(self, sample: tuple[torch.Tensor, torch.Tensor]):
        normalized_sample = (sample[0] - self.min_val) / (self.max_val - self.min_val), sample[1]
        return normalized_sample

### <a id='toc2_3_1_'></a>[Direct transform](#toc0_)


In [ ]:
# dataset
X = np.array([[1, 2, 3], [5, 1, 2], [3, 3, 3]])
y = np.array([[0], [0], [1]])
dataset = list(zip(X, y))

# log
print(f"dataset : {dataset}")

In [ ]:
# NumpyToTensor
t_totensor = NumpyToTensor()

# NormalizeTo01
t_normalize = NormalizeTo01()
t_normalize.fit(X)

# transform the first input
result = t_normalize(t_totensor(dataset[0]))

# log
print(f"result          : {result}")
print(f"result[0].dtype : {result[0].dtype}")
print(f"result[1].dtype : {result[1].dtype}")

### <a id='toc2_3_2_'></a>[Integrated transform](#toc0_)


In [11]:
# advanced Dataset with transform support
class AdvancedCustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

        # if NormalizeTo01 is included, call <fit> method for that
        if self.transform:
            for t in self.transform.transforms:
                if isinstance(t, NormalizeTo01):
                    t.fit(self.data)
                    break

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index], self.labels[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
X = np.array([[1, 2, 3], [5, 1, 2], [3, 3, 3]])
y = np.array([[0], [0], [1]])

transformations = transforms.Compose(
    [
        NumpyToTensor(),
        NormalizeTo01(),
    ]
)

dataset = AdvancedCustomDataset(X, y, transformations)

# log
for i in range(len(y)):
    print(f"dataset[{i}]: {dataset[i]}")
    print(f"    -> input data : {dataset[i][0]}")
    print(f"    -> label      : {dataset[i][1]}", end="\n\n")

## <a id='toc2_4_'></a>[Custom Activation Function](#toc0_)

- you can create your own activation function by subclassing `torch.nn.Module`.
- Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:

- Non-linear Activations (weighted sum, nonlinearity): [pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
- Non-linear Activations (other): [pytorch.org/docs/stable/nn.html#non-linear-activations-other](https://pytorch.org/docs/stable/nn.html#non-linear-activations-other)
- Non-linear activation functions: [pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions)


In [13]:
class CustomSigmoid(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.sigmoid(x)

In [ ]:
sig_1 = CustomSigmoid()
sig_2 = nn.Sigmoid()

values = torch.tensor([10, 0, -10], dtype=torch.float32)

# log
print(f"sig_1(values) : {sig_1(values)}")
print(f"sig_2(values) : {sig_2(values)}")

## <a id='toc2_5_'></a>[Custom Model](#toc0_)

- Sequential Model:
  - Useful for simpler models where the layers are stacked in a linear sequence
  - The `torch.nn.Sequential` class allows you to stack layers in a sequence, passing the output of one layer directly to the next.
  - This is great for simple models like fully-connected neural networks or basic CNNs.
  - Key Points
    - Layers are defined in the order they are passed to `Sequential`.
    - You don't need to define the `forward` method manually; PyTorch handles it for you.
- Functional Model:
  - Allowing for complex architectures where you might need non-linear layer connections (e.g., skip connections in ResNet)
  - models are created by subclassing `torch.nn.Module`.
  - This allows you to define any neural network architecture, from simple feedforward networks to complex architectures like GANs or transformers
  - Key Points
    - Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:

- Module: [pytorch.org/docs/stable/generated/torch.nn.Module.html](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- torch.nn: [pytorch.org/docs/stable/nn.html](https://pytorch.org/docs/stable/nn.html)
- Building Models with PyTorch: [pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html)
- Build the Neural Network: [pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
- Neural Networks: [pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial)


In [ ]:
# a simple sequential model
model_1 = nn.Sequential(
    nn.Linear(in_features=30, out_features=16), nn.Sigmoid(), nn.Linear(in_features=16, out_features=1), nn.Sigmoid()
)

# log
model_1

In [ ]:
# a simple functional model
class CustomLogisticRegression(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super().__init__()

        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.sigmoid1 = CustomSigmoid()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.sigmoid2 = CustomSigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        return x


# initialize the model
model_2 = CustomLogisticRegression(30, 16, 1)

# log
model_2

## <a id='toc2_6_'></a>[Custom Loss Function](#toc0_)

- PyTorch comes with standard loss functions like MSE, Cross-Entropy, etc.
- you can create your own loss function by subclassing `torch.nn.Module`.
- Use `torch.nn.Module` as the parent class and implement  `forward` method.

📝 **Docs & Tutorials** 📚:

- Loss Functions: [pytorch.org/docs/stable/nn.html#loss-functions](https://pytorch.org/docs/stable/nn.html#loss-functions)


In [17]:
class CustomMSE(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_pred, y_true):
        loss = torch.mean((y_pred - y_true) ** 2)
        return loss

In [ ]:
# split dataset into (data, labels)
X = dataset_1[:][0]
y_true = dataset_1[:][1]

# feed-forward
y_pred = model_1(X)

# MSE loss function
criterion_1 = CustomMSE()  # custom
criterion_2 = nn.MSELoss()  # built-in

# compute the loss
loss_1 = criterion_1(y_pred, y_true)
loss_2 = criterion_2(y_pred, y_true)

# log
print(f"loss_1: {loss_1}")
print(f"loss_2: {loss_2}")

## <a id='toc2_7_'></a>[Custom Optimizer](#toc0_)

- PyTorch offers optimizers like SGD, Adam, etc.
- you can create your own optimizer by subclassing `torch.optim.Optimizer`.
- Use `torch.optim.Optimizer` as the parent class and override `step` method.

📝 **Docs & Tutorials** 📚:

- torch.optim: [pytorch.org/docs/stable/optim.html](https://pytorch.org/docs/stable/optim.html)
- torch.optim.Optimizer.step: [pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html)


In [19]:
# this implementation might not be the same as SGD
class CustomSGD(optim.Optimizer):
    def __init__(self, params, lr=0.01, momentum=0):
        defaults = dict(lr=lr, momentum=momentum)
        super().__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            momentum = group["momentum"]

            for p in group["params"]:
                if p.grad is None:
                    continue
                d_p = p.grad.data

                if momentum != 0:
                    param_state = self.state[p]
                    if "momentum_buffer" not in param_state:
                        buf = param_state["momentum_buffer"] = torch.zeros_like(p.data)
                        buf.mul_(momentum).add_(d_p)
                    else:
                        buf = param_state["momentum_buffer"]
                        buf.mul_(momentum).add_(d_p, alpha=1 - momentum)
                    d_p = buf

                p.data.add_(d_p, alpha=-lr)

In [ ]:
optimizer_1 = CustomSGD(model_1.parameters())
optimizer_2 = optim.SGD(model_1.parameters())

# log
print(f"optimizer_1:\n{optimizer_1}\n")
print(f"optimizer_2:\n{optimizer_2}")

# <a id='toc3_'></a>[Example: All In One](#toc0_)


In [21]:
# load breast-cancer dataset
X, y = df.iloc[:, 2:].values, df.iloc[:, 1].values


# create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]


# convert numpy.ndarray to torch.Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# create a dataset
dataset = CustomDataset(X, y)

# create a dataloader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# custom sigmoid activation
class CustomSigmoid(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.sigmoid(x)


# model
class CustomLogisticRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.sigmoid1 = CustomSigmoid()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.sigmoid2 = CustomSigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        return x


input_size = X.shape[1]
hidden_size = 2
output_size = y.shape[1]

# model
model = CustomLogisticRegression(input_size, hidden_size, output_size)
model

In [23]:
# hyper parameters
epochs = 10
lr = 0.005
criterion = CustomMSE()
optimizer = CustomSGD(model.parameters(), lr=lr)

In [ ]:
# training loop
model.train()
total_loss = []
total_acc = []

for epoch in range(epochs):

    epoch_loss = 0
    epoch_acc = 0

    for x, y_true in dataloader:

        # forward
        y_pred = model(x)
        loss = criterion(y_pred, y_true)

        # backward
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

        # store loss and accuracy per iteration
        epoch_loss += loss.item()
        epoch_acc += ((y_pred > 0.5).float() == y_true).sum().item()

    total_loss.append(epoch_loss / len(dataloader))
    total_acc.append(epoch_acc / len(X))

    # log
    print(
        f"epoch {epoch+1:0{len(str(epochs))}}/{epochs} -> loss: {total_loss[-1]:.5f} - accuracy: {total_acc[-1]*100:.2f}%"
    )